<a href="https://colab.research.google.com/github/SeokcheonMoon/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unVar_second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime 이후 상단바 런타임의 Restart Session 을 클릭해야함
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import scipy
import scipy.stats as stats

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
df_fruitstore = pd.read_csv("/content/drive/MyDrive/fruitstore_saleshistory.csv")
df_fruitstore

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05
2,2018-01-02,화,1,1,1506656256,정상회원,축산물,알,유정란,유정란/매장용(10알/국내산),1.0,수지구,풍덕천동,여,36.0,30대이하,7083,매장C,NaN,10:05
3,2018-01-02,화,1,1,1023108864,정상회원,반찬,두부/유부,두부,연두부(100g),1.0,수지구,풍덕천동,여,36.0,30대이하,766,매장C,NaN,10:08
4,2018-01-02,화,1,1,1476143616,정상회원,간식,빵,"식빵, 식사대용",아침빵(240g/8개)/매장,1.0,수지구,풍덕천동,여,34.0,30대이하,4403,매장C,NaN,10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705566,2018-06-30,토,6,26,146297344,정상회원,수산,건어물,멸치류,국물멸치(300g),1.0,수지구,성복동,여,51.0,50대,9189,매장A,NaN,17:56
705567,2018-06-30,토,6,26,1532985472,정상회원,수산,해조,미역/다시마,자른다시마(100g),1.0,수지구,동천동,여,63.0,60대,5169,매장A,NaN,14:37
705568,2018-06-30,토,6,26,1532985472,정상회원,반찬,두부/유부,두부,두부(1모:420g),2.0,수지구,동천동,여,63.0,60대,7466,매장A,NaN,14:37
705569,2018-06-30,토,6,26,1209779776,정상회원,간식,과자,스낵,하늘바다새우(60g),1.0,수지구,동천동,여,41.0,40대,3542,매장A,NaN,18:04


## 가설 : 고객의 연령대별 구매한 과일 종류의 평균 가격이 이분산을 보이는지?

- 귀무가설 : 고객의 연령대에 따른 과일의 평균 가격은 이분산을 보이지 않는다.(고객의 연령대와 과일의 평균 가격은 관련이 없다.)
- 대립가설 : 고객의 연령대에 따른 과일의 평균 가격은 이분산을 보인다.(고객의 연령대와 과일의 평균 가격은 관련이 있다.)

### 정규/비정규 분포 판단 - Anderson test


In [3]:
# 레코드 갯수가 5천개 이상 Anderson test
stats.anderson(df_fruitstore["구매금액"])

AndersonResult(statistic=80887.23714091501, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]), fit_result=  params: FitParams(loc=10920.038082630947, scale=16703.23407010683)
 success: True
 message: '`anderson` successfully fit the distribution to the data.')

statistic > critical_values 이므로 => 비정규분포를 따른다.

### 등분산/비분산 판단 - Kruskal test(범주(연령대)가 3가지이상)

In [4]:
condition_fruit = "물품중분류=='과일'"
df_fruitstore_fruit = df_fruitstore.query(condition_fruit)
df_fruitstore_fruit

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
62,2018-01-02,화,1,1,445131904,정상회원,과실,과일,건대추,건대추(500g),1.0,수지구,풍덕천동,여,58.0,50대,22973,매장C,NaN,12:18
132,2018-01-02,화,1,1,744508416,정상회원,과실,과일,귤(조생),귤/유(1.5kg),1.0,기흥구,보정동,여,63.0,60대,11104,매장C,NaN,14:58
187,2018-01-02,화,1,1,281649856,정상회원,과실,과일,귤(조생),귤/유(1.5kg),1.0,수지구,풍덕천동,여,41.0,40대,11104,매장C,NaN,16:49
220,2018-01-02,화,1,1,548171456,정상회원,과실,과일,배,배(대:3개),1.0,수지구,풍덕천동,여,52.0,50대,16847,매장C,NaN,18:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705342,2018-06-30,토,6,26,862820992,정상회원,과실,과일,황매실,황매실(남고)/유(5kg),-2.0,수지구,성복동,남,53.0,50대,-111036,매장D,20180629.0,13:55
705354,2018-06-30,토,6,26,1553271552,정상회원,과실,과일,황매실,황매실(남고)/유(5kg),2.0,수지구,신봉동,여,45.0,40대,111036,매장D,NaN,12:49
705429,2018-06-30,토,6,26,606984384,정상회원,과실,과일,산딸기,냉동산딸기(250g),3.0,수지구,풍덕천동,여,51.0,50대,35608,매장B,NaN,19:34
705467,2018-06-30,토,6,26,1145758912,정상회원,과실,과일,사과,사과/(2kg)/소,1.0,수지구,상현동,여,37.0,30대이하,13592,매장B,NaN,18:11


In [5]:
df_fruitstore_fruit["연령대"].unique()

array(['40대', '50대', '60대', '70대이상', '30대이하'], dtype=object)

In [6]:
df_fruitstore_fruit["연령대"].dropna()

0           40대
62          50대
132         60대
187         40대
220         50대
          ...  
705342      50대
705354      40대
705429      50대
705467    30대이하
705478      40대
Name: 연령대, Length: 15402, dtype: object

In [7]:
df_fruitstore_fruit["연령대"].value_counts()

40대      5208
50대      4052
60대      2444
30대이하    1888
70대이상    1810
Name: 연령대, dtype: int64

In [8]:
age_list =  df_fruitstore_fruit["연령대"].value_counts().index.to_list()
age_list

['40대', '50대', '60대', '30대이하', '70대이상']

In [9]:
series_list = list()
for age in age_list :
  condition = f"연령대 == '{age}'"
  series_price = df_fruitstore.query(condition)["구매금액"]
  series_list.append(series_price)
series_list

[0         22207
 17         5743
 18         8711
 19         5169
 20         5265
           ...  
 705556     2202
 705560     7275
 705561     3982
 705569     3542
 705570     1914
 Name: 구매금액, Length: 270573, dtype: int64,
 5          7083
 6         17038
 7         11104
 8          7083
 9          2584
           ...  
 705548     2680
 705549     1819
 705557     5169
 705562    77725
 705566     9189
 Name: 구매금액, Length: 174458, dtype: int64,
 12         7849
 13        10529
 42         7083
 44         2489
 63         3733
           ...  
 705558    11678
 705559     8615
 705565    -6126
 705567     5169
 705568     7466
 Name: 구매금액, Length: 95019, dtype: int64,
 1          4977
 2          7083
 3           766
 4          4403
 10         7179
           ...  
 705532     8232
 705543    12444
 705553     4977
 705563     5743
 705564     3446
 Name: 구매금액, Length: 106392, dtype: int64,
 15        58773
 16        15507
 30        16273
 41         7083
 53         3

In [10]:
stats.kruskal(series_list[0],series_list[1],series_list[2],series_list[3],series_list[4])

KruskalResult(statistic=1585.643271898882, pvalue=0.0)

## 결과
- 통계결과 : pvalue < 0.05 이므로 대립가설을 따른다.
- 분석결과 : 고객의 연령대에 따른 과일의 평균 가격은 이분산을 보인다.